In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="subprocess.run")

%run utility_functions.ipynb

In [2]:
df_original = pd.read_csv("data/Original_Preprocessed_data.csv")
df_original.shape

(1547, 25)

# Train test split

In [3]:
from sklearn.model_selection import train_test_split

X = df_original.drop('Healthy', axis=1)
y = df_original['Healthy']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [4]:
X_train.shape

(1237, 24)

In [5]:
y_train.value_counts()

Healthy
1    1183
0      54
Name: count, dtype: int64

# SMOTE

In [6]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

# Upsample the minority class
X_train, y_train = smote.fit_resample(X_train, y_train)

  File "C:\Users\younu\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\younu\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 546, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\younu\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1022, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\younu\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1491, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [7]:
# X_test.to_csv("X_test.csv", index=None)
# y_test.to_csv("y_test.csv", index=None)

# Scaling

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import f1_score, classification_report

rf_classifier = LogisticRegression(random_state=42)
rf_classifier.fit(X_train, y_train)
print("============================= Test result ===========================")
y_pred = rf_classifier.predict(X_test)
evaluate_classifier(y_test, y_pred) 

============================= Test result ===========================
Accuracy: 0.9451612903225807
AUC Score: 0.9346024346024346
F1 Score: 0.9706390328151986
Specificity: 0.9230769230769231
Sensitivity: 0.9461279461279462
Classification Report:
              precision    recall  f1-score   support

           0       0.43      0.92      0.59        13
           1       1.00      0.95      0.97       297

    accuracy                           0.95       310
   macro avg       0.71      0.93      0.78       310
weighted avg       0.97      0.95      0.95       310



# RandomForest

In [10]:
from sklearn.ensemble import RandomForestClassifier

clf_random = RandomForestClassifier(n_estimators=300, random_state=42)
clf_random.fit(X_train, y_train)

print("============================= Test result ===========================")
y_pred = clf_random.predict(X_test)
evaluate_classifier(y_test, y_pred) 

============================= Test result ===========================
Accuracy: 0.9967741935483871
AUC Score: 0.9615384615384616
F1 Score: 0.9983193277310924
Specificity: 0.9230769230769231
Sensitivity: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        13
           1       1.00      1.00      1.00       297

    accuracy                           1.00       310
   macro avg       1.00      0.96      0.98       310
weighted avg       1.00      1.00      1.00       310



In [11]:
from pytorch_tabnet.tab_model import TabNetClassifier

clf_tabnet = TabNetClassifier()
clf_tabnet.fit(
  X_train, y_train,
  eval_set=[(X_test, y_test)],
  eval_metric=['auc'],
    max_epochs = 300,
    batch_size=512,
    patience=100,
)

epoch 0  | loss: 0.82238 | val_0_auc: 0.69412 |  0:00:00s
epoch 1  | loss: 0.46992 | val_0_auc: 0.75136 |  0:00:00s
epoch 2  | loss: 0.30317 | val_0_auc: 0.87231 |  0:00:00s
epoch 3  | loss: 0.22504 | val_0_auc: 0.84745 |  0:00:00s
epoch 4  | loss: 0.17766 | val_0_auc: 0.90339 |  0:00:01s
epoch 5  | loss: 0.13745 | val_0_auc: 0.90417 |  0:00:01s
epoch 6  | loss: 0.12633 | val_0_auc: 0.94457 |  0:00:01s
epoch 7  | loss: 0.10482 | val_0_auc: 0.91608 |  0:00:01s
epoch 8  | loss: 0.09595 | val_0_auc: 0.91997 |  0:00:02s
epoch 9  | loss: 0.09239 | val_0_auc: 0.92023 |  0:00:02s
epoch 10 | loss: 0.08058 | val_0_auc: 0.91453 |  0:00:02s
epoch 11 | loss: 0.08124 | val_0_auc: 0.90106 |  0:00:03s
epoch 12 | loss: 0.07816 | val_0_auc: 0.91142 |  0:00:03s
epoch 13 | loss: 0.06422 | val_0_auc: 0.85341 |  0:00:03s
epoch 14 | loss: 0.07015 | val_0_auc: 0.856   |  0:00:03s
epoch 15 | loss: 0.06633 | val_0_auc: 0.86506 |  0:00:04s
epoch 16 | loss: 0.06488 | val_0_auc: 0.98601 |  0:00:04s
epoch 17 | los

In [12]:
y_pred = clf_tabnet.predict(X_test)
evaluate_classifier(y_test, y_pred) 

Accuracy: 0.9935483870967742
AUC Score: 0.9598549598549598
F1 Score: 0.9966329966329966
Specificity: 0.9230769230769231
Sensitivity: 0.9966329966329966
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        13
           1       1.00      1.00      1.00       297

    accuracy                           0.99       310
   macro avg       0.96      0.96      0.96       310
weighted avg       0.99      0.99      0.99       310



## TabFPN

In [13]:
# from tabpfn import TabPFNClassifier

# classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)

# classifier.fit(X_train, y_train)
# y_pred, p_eval = classifier.predict(X_test, return_winning_probability=True)

In [14]:
# y_pred = classifier.predict(X_test)
# evaluate_classifier(y_test, y_pred) 

# LSTM

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape input data to fit the LSTM layer
# LSTM expects input data in the shape (samples, timesteps, features)
# Here, assuming 23 features
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Creating the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.2))  # Optional dropout layer for regularization
model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation for binary classification

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=256, validation_data=(X_test_reshaped, y_test))

Epoch 1/100
10/10 [==============================] - 8s 189ms/step - loss: 0.6597 - accuracy: 0.7540 - val_loss: 0.6232 - val_accuracy: 0.8387
Epoch 2/100
10/10 [==============================] - 0s 30ms/step - loss: 0.5950 - accuracy: 0.8762 - val_loss: 0.5728 - val_accuracy: 0.8387
Epoch 3/100
10/10 [==============================] - 0s 20ms/step - loss: 0.5402 - accuracy: 0.8935 - val_loss: 0.5212 - val_accuracy: 0.8484
Epoch 4/100
10/10 [==============================] - 0s 21ms/step - loss: 0.4913 - accuracy: 0.8935 - val_loss: 0.4794 - val_accuracy: 0.8484
Epoch 5/100
10/10 [==============================] - 0s 20ms/step - loss: 0.4438 - accuracy: 0.9024 - val_loss: 0.4365 - val_accuracy: 0.8645
Epoch 6/100
10/10 [==============================] - 0s 21ms/step - loss: 0.3998 - accuracy: 0.9112 - val_loss: 0.3978 - val_accuracy: 0.8774
Epoch 7/100
10/10 [==============================] - 0s 21ms/step - loss: 0.3596 - accuracy: 0.9134 - val_loss: 0.3630 - val_accuracy: 0.8871
Epoch

In [26]:
y_pred = model.predict(X_test_reshaped)
y_pred = (y_pred >= .5).astype(int)

10/10 [==============================] - 1s 4ms/step


In [27]:
evaluate_classifier(y_test, y_pred)

Accuracy: 0.9709677419354839
AUC Score: 0.8377363377363378
F1 Score: 0.984822934232715
Specificity: 0.6923076923076923
Sensitivity: 0.9831649831649831
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.69      0.67        13
           1       0.99      0.98      0.98       297

    accuracy                           0.97       310
   macro avg       0.81      0.84      0.83       310
weighted avg       0.97      0.97      0.97       310

